In [2]:
import os
import json
import numpy as np
import pandas as pd
import evaluate
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms as T
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from transformers import AutoImageProcessor, MobileNetV2Model, AutoModelForSemanticSegmentation
from datasets import load_dataset, Dataset, DatasetDict, Image
PATH_DATASETS = '/home/morten/Documents/Algorithms-for-Automated-Driving/code/solutions/lane_detection/data_lane_segmentation/'

/home/morten/mambaforge/envs/Hug/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
img_dir = PATH_DATASETS + 'train/'
label_dir = PATH_DATASETS + 'train_label/'

image_path = []
for dirname, _, filenames_fit in os.walk(img_dir):
    for filename_fit in filenames_fit:
        image_path.append(filename_fit.split('.')[0])

label_path = []
for dirname, _, filenames in os.walk(label_dir):
    for filename in filenames:
        label_path.append(filename.split('.')[0])

img_fit = pd.DataFrame({'id': image_path}, index = np.arange(0, len(image_path)))
label_fit = pd.DataFrame({'id': label_path}, index = np.arange(0, len(label_path)))

lane_train_image_path, lane_val_image_path = train_test_split(img_fit['id'].values, test_size=0.2, train_size=0.8, random_state=10)

lane_train_label_path = np.copy(lane_train_image_path)

lane_val_label_path = np.copy(lane_val_image_path)


for i in range(0, len(lane_train_image_path)):
    lane_train_image_path[i] = img_dir + lane_train_image_path[i] + '.png'
    lane_train_label_path[i] = label_dir + lane_train_label_path[i] + '_label.png'

for i in range(0, len(lane_val_image_path)):
    lane_val_image_path[i] = img_dir + lane_val_image_path[i] + '.png'
    lane_val_label_path[i] = label_dir + lane_val_label_path[i] + '_label.png'

def create_dataset(img_paths, lab_paths):
    dataset = Dataset.from_dict({'image': img_paths,
                                 'label': lab_paths})
    dataset = dataset.cast_column('image', Image())
    dataset = dataset.cast_column('label', Image())

    return dataset

train_dataset = create_dataset(lane_train_image_path, lane_train_label_path)
val_dataset = create_dataset(lane_val_image_path, lane_val_label_path)

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
})

dataset.push_to_hub('lane_master')

Pushing split train to the Hub.
Pushing dataset shards to the dataset hub: 100%|██████████| 4/4 [11:36<00:00, 174.11s/it]
Pushing split validation to the Hub.
Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [03:51<00:00, 231.03s/it]


3


In [16]:
from huggingface_hub import HfApi

api = HfApi()

id2label = {0: 'background', 1: 'left', 2: 'right'}
with open('id2label.json', 'w') as fp:
    json.dump(id2label, fp)

print(id2label)

id2label = {int(k): v for k, v in id2label.items()}
print(id2label)

label2id = {v: k for k, v in id2label.items()}
print(label2id)

num_labels = len(id2label)

api.upload_file(
    path_or_fileobj="id2label.json",
    path_in_repo="id2label.json",
    repo_id="Efferbach/lane_master",
    repo_type="dataset",
)


{0: 'background', 1: 'left', 2: 'right'}
{0: 'background', 1: 'left', 2: 'right'}
{'background': 0, 'left': 1, 'right': 2}


'https://huggingface.co/datasets/Efferbach/lane_master/blob/main/id2label.json'